# Object Detection for Face Detection Project

This notebook demonstrates the use of pre-trained object detection models, specifically focusing on detecting faces in images and live video feeds. The goal of this project is to implement and compare different object detection models, starting with YOLO (You Only Look Once), and assess their performance in terms of accuracy and speed.

## Objectives:
1. Research and select a suitable pre-trained object detection model (e.g., YOLOv5 or YOLOv8).
2. Validate the model using a sample dataset to assess its performance.
3. Implement the model to detect faces in static images and output annotated images with bounding boxes.
4. Extend the functionality to handle live video feeds from a webcam, with real-time face detection and bounding box visualization.

Throughout the notebook, we will evaluate key metrics such as accuracy (mAP) and inference speed (FPS), making adjustments as needed to optimize for both performance and accuracy.


### Import necessary libraries

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO
import os
import cv2
import zipfile
import random
import shutil

### Create Folders

In [34]:
if not os.path.exists('Training'):
    os.makedirs('Training')

if not os.path.exists('Validation'):
    os.makedirs('Validation')

if not os.path.exists('Training/labels'):
    os.makedirs('Training/labels')

if not os.path.exists('Validation/labels'):
    os.makedirs('Validation/labels')

if not os.path.exists('Training/images'):
    os.makedirs('Training/images')

if not os.path.exists('Validation/images'):
    os.makedirs('Validation/images')



### Unzip data

In [35]:
def unzip_images_only(zip_file_path, dest_dir, images_location):
    """
    Unzips only the 'images' folder from a given zip file to the specified destination directory.
    It extracts all images from the 'WIDER_train/images' folder and places them directly into 'dest_dir/images'.

    Parameters:
    zip_file_path (str): Path to the zip file to be extracted.
    dest_dir (str): Directory where the images should be extracted, into a subdirectory named 'images'.
    images_location (str): The folder path inside the zip file where images are located.
    """
    # Define the full path for the 'images' subdirectory
    images_dest_dir = os.path.join(dest_dir, 'images')

    # Ensure the 'images' subdirectory exists
    if not os.path.exists(images_dest_dir):
        os.makedirs(images_dest_dir)

    # Open the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            # Extract only files from the specified 'images_location' folder
            if file_info.filename.startswith(images_location) and not file_info.is_dir():
                # Flatten the structure by moving all images directly into 'images_dest_dir'
                # Extract the filename only (ignore subdirectories)
                filename = os.path.basename(file_info.filename)
                target_path = os.path.join(images_dest_dir, filename)
                
                # Extract the file to the target path
                with zip_ref.open(file_info) as source, open(target_path, 'wb') as target:
                    target.write(source.read())

        print(f"Extracted all images from '{zip_file_path}' to '{images_dest_dir}' successfully, with flattened directory structure.")


# Unzip train data
zip_file_path_train = 'WIDER_train.zip'
dest_dir_train = 'Training'
images_location_train = 'WIDER_train/images/'

if not os.listdir('Training/images'):
    unzip_images_only(zip_file_path_train, dest_dir_train, images_location_train)

# Unzip validation data
zip_file_path_val = 'WIDER_val.zip'
dest_dir_val = 'Validation'
images_location_val = 'WIDER_val/images/'

if not os.listdir('Validation/images'):
    unzip_images_only(zip_file_path_val, dest_dir_val, images_location_val)

Extracted all images from 'WIDER_train.zip' to 'Training\images' successfully, with flattened directory structure.
Extracted all images from 'WIDER_val.zip' to 'Validation\images' successfully, with flattened directory structure.


### Unzip GT data

In [16]:
def unzip_file(zip_file_path, dest_dir="."):
    """
    Unzips a given zipped file to the specified destination directory.

    Parameters:
    zip_file_path (str): Path to the zip file to be extracted.
    dest_dir (str): Directory where the zip file should be extracted.
                    Default is the current working directory.
    """
    # Check if the destination directory exists, if not, create it
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Unzipping the file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(dest_dir)
        print(f"Extracted '{zip_file_path}' to '{os.path.abspath(dest_dir)}' successfully.")

# Example usage
zip_file_path = 'wider_face_split.zip'

if not os.path.exists(zip_file_path):
    unzip_file(zip_file_path)

### Convert Annotations From the Wider Format to the YOLO Format

In [38]:

# Function to parse annotations and convert them to YOLO format
def convert_to_yolo_format(annotations_file, images_root, output_labels_dir):
    with open(annotations_file, 'r') as file:
        lines = file.readlines()

    i = 0
    while i < len(lines):
        # Parse image path
        image_path = lines[i].strip()

        # Skip if this line does not represent an image path
        if not image_path.endswith('.jpg'):
            i += 1
            continue

        # Extract only the filename (ignore any subdirectory in the annotation)
        image_filename = os.path.basename(image_path)
        image_full_path = os.path.join(images_root, image_filename)

        # Skip if image does not exist
        if not os.path.exists(image_full_path):
            print(f"Warning: Image {image_full_path} does not exist.")
            i += 1
            # Skip to the next image by finding the next valid image path
            while i < len(lines) and not lines[i].strip().endswith('.jpg'):
                i += 1
            continue

        # Load image to get dimensions
        image = cv2.imread(image_full_path)
        if image is None:
            print(f"Error: Failed to load image {image_full_path}")
            i += 1
            # Skip to the next image by finding the next valid image path
            while i < len(lines) and not lines[i].strip().endswith('.jpg'):
                i += 1
            continue

        image_height, image_width, _ = image.shape

        # Parse number of bounding boxes
        try:
            num_boxes = int(lines[i + 1].strip())
        except ValueError:
            print(f"Error: Expected an integer for number of bounding boxes but got: {lines[i + 1].strip()}")
            i += 1
            # Skip to the next image by finding the next valid image path
            while i < len(lines) and not lines[i].strip().endswith('.jpg'):
                i += 1
            continue

        # Create the label file path in the output labels directory
        label_path = os.path.join(output_labels_dir, image_filename.replace('.jpg', '.txt'))

        with open(label_path, 'w') as label_file:
            # Loop through bounding boxes
            for j in range(num_boxes):
                try:
                    box_data = list(map(int, lines[i + 2 + j].split()[:4]))  # Extract x, y, width, height
                    x, y, width, height = box_data

                    # Convert to YOLO format
                    x_center = (x + width / 2) / image_width
                    y_center = (y + height / 2) / image_height
                    width_normalized = width / image_width
                    height_normalized = height / image_height

                    # Write to label file in YOLO format
                    label_file.write(f"0 {x_center} {y_center} {width_normalized} {height_normalized}\n")
                except ValueError:
                    print(f"Warning: Failed to parse bounding box for {image_filename}, skipping this bounding box.")
                    continue

        # Move to the next image's annotations
        i += 2 + num_boxes

# Define paths

# Training Data
annotations_file_training = 'wider_face_split/wider_face_train_bbx_gt.txt'
images_root_training = 'Training/images'
output_labels_dir_training = 'Training/labels'

if not os.path.exists(output_labels_dir_training):
    os.makedirs(output_labels_dir_training)

# Validation Data
annotations_file_validation = 'wider_face_split/wider_face_val_bbx_gt.txt'
images_root_validation = 'Validation/images'
output_labels_dir_validation = 'Validation/labels'

if not os.path.exists(output_labels_dir_validation):
    os.makedirs(output_labels_dir_validation)

# Convert the annotations to YOLO format
if not os.listdir(output_labels_dir_training):
    convert_to_yolo_format(annotations_file_training, images_root_training, output_labels_dir_training)

if not os.listdir(output_labels_dir_validation):
    convert_to_yolo_format(annotations_file_validation, images_root_validation, output_labels_dir_validation)


### Load Pre-Trained YOLO v8n model

In [39]:
model = YOLO('yolov8n.pt')

### Verify model

In [40]:
# Load your YOLO model
model = YOLO('yolov8n.pt')

# Define the path to the images directory
images_dir = 'Training/images/'

# List all image files in the directory
image_files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_dir) for f in filenames if f.endswith(('.jpg'))]

# Ensure there are images available
if len(image_files) == 0:
    print("No images found in the directory.")
else:
    # Randomly select an image from the list
    random_image = random.choice(image_files)
    print(f"Randomly selected image: {random_image}")

    # Run inference on the selected image
    results = model(random_image)

    # Display the results
    results[0].show()

Randomly selected image: Training/images/35_Basketball_basketballgame_ball_35_158.jpg

image 1/1 C:\Users\Alfred Aronsson\SSY340-Project\object-detection\Training\images\35_Basketball_basketballgame_ball_35_158.jpg: 640x480 2 persons, 335.2ms
Speed: 8.8ms preprocess, 335.2ms inference, 4.4ms postprocess per image at shape (1, 3, 640, 480)


### Train model

In [43]:
model.train(data='data.yaml', epochs=50, batch=16, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.14 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.13  Python-3.11.9 torch-2.3.1 CPU (Intel Core(TM) i5-1035G7 1.20GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_tx

100%|███████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 23.8MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning C:\Users\Alfred Aronsson\SSY340-Project\object-detection\Training\labels... 280 images, 12601 backgroun


train: New cache created: C:\Users\Alfred Aronsson\SSY340-Project\object-detection\Training\labels.cache


val: Scanning C:\Users\Alfred Aronsson\SSY340-Project\object-detection\Validation\labels... 3226 images, 0 backgrounds,

val: WARNING  C:\Users\Alfred Aronsson\SSY340-Project\object-detection\Validation\images\21_Festival_Festival_21_604.jpg: 1 duplicate labels removed
val: WARNING  C:\Users\Alfred Aronsson\SSY340-Project\object-detection\Validation\images\39_Ice_Skating_iceskiing_39_583.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0019531]


val: New cache created: C:\Users\Alfred Aronsson\SSY340-Project\object-detection\Validation\labels.cache
Plotting labels to runs\detect\train3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 63 weight(decay=0.0), 70 weight(decay=0.0005), 69 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      3.544      60.43       1.53          6        640:   1%|          | 10/805 [02:45<3:39:04, 


KeyboardInterrupt: 